In [93]:
import pandas as pd
import plotly.express as px

# Oh Hell

In case you aren't familiar, oh hell is a trick taking game a little bit like hearts. The major difference is that at the start of each round, each participant guesses how many tricks they will take. You only get points if you get *exactly* that many tricks. Another peculiarity of the game is that the size of the hand changes each round, starting at ten and going down by one till you are dealt just a single card. 

Let's take a look at the one card hand as a nice pocket example and try to figure out the optimal play. Let's assume you've dealt (which means you're leading) You have a binary deciscion to make- you can guess one, saying you will take the trick, or 0 saying that you won't take the trick. You have three peices of information- the card in your hand, how many other players there are, and one card you know is out of the game (one card is flipped over to select the trump at the start of each round).

To beat your card, another player would have to have a card of the same suite bigger than your card or a trump or a higher trump if the card in your hand is a trump. We can boil the question down to something more mathematically tractable- what is the probability that someone else at the table has a card that will beat your card? You should bet 1 if there is a more than 50% chance you will win the hand, otherwise 0. 

Let's specify some numbers, we'll value the face cards as 11 for the jack, 12 for the queen, etc. 

In [16]:
deck_size = 52
num_players = 3
trump_display_card_val = 11
trump_suite = "hearts"
your_card_value = 9
your_card_suite = "diamonds"

Okay, so based on all the trick taking rules we know two full suites won't take your card. Spades and clubs won't take the trick, and neither will the 2 of diamonds through the 8 of diamonds. That's a total of 33 cards (26 of the other suites, 7 of the lower cards in the same suite) that won't take the trick. There are 50 cards in the deck, and we dealt 2 of them to the other players. 

Well, for the first player we know they have a 33/50 chance of not picking a card that's bigger than our card. Assuming that's happend the second player has a 32/49 chance of not picking a card that's bigger than our card. Multiplying those probabilities we get:

In [184]:
33/50 * 32/49

0.4310204081632653

So you should guess 0 in this scenario!

Let's see if we can generalize this solution. I'd like to know what the lowest card you should bet 1 on is for each number of players. 

In [61]:
def get_num_cards_that_take_trick(tcard_out_val, is_trump, in_hand_val):
    # This is pretty easy if you have trump- it's just how many trump cards are higher 
    if is_trump:
        bigger_trumps = 14 - in_hand_val 
        if tcard_out_val > in_hand_val: # If the trump card out is bigger, we take one away here
            bigger_trumps = bigger_trumps - 1
        return bigger_trumps
    
    n_trumps = 12 # one is flipped at the start of the round to define trump class
    bigger_in_hand_suite = 14 - in_hand_val 
    return n_trumps + bigger_in_hand_suite

In [137]:
def probability_of_winning_trick(tcard_out_val, is_trump, in_hand_val, num_players):
    num_cards_that_take = get_num_cards_that_take_trick(tcard_out_val, is_trump, in_hand_val)
    probability_of_taking = 1
    for n in range(num_players - 1):
        prob_no_take = ((50 - num_cards_that_take - n) / (50 - n))
        probability_of_taking = probability_of_taking * prob_no_take
    return probability_of_taking

In [138]:
for n in range(2):
    print(n)

0
1


In [139]:
probability_of_winning_trick(14, False, 9, 3)

0.4310204081632653

In [140]:
probability_of_winning_trick(14, False, 9, 4)

0.2783673469387755

In [141]:
probability_of_winning_trick(14, False, 9, 5)

0.1776812852800695

In [171]:
values = range(2, 15) 
is_trump = [False, True]
players = range(2, 8)
trump_card_vals = range(2, 15)
results = []

In [172]:
for player_num in players:
    for trump in is_trump:
        for your_card_val in values:
            for trump_card_val in trump_card_vals:
                prob_take = probability_of_winning_trick(trump_card_val, trump, your_card_val, player_num)
                results.append( {
                    'probability_take' : prob_take,
                    'is_trump' : trump,
                    'card_value' : your_card_val,
                    'trump_card_val' : trump_card_val,
                    'players' : player_num
                })

In [173]:
prob_win_df = pd.DataFrame(results)

In [174]:
prob_win_df

,probability_take,is_trump,card_value,trump_card_val,players
0,0.52,False,2,2,2
1,0.52,False,2,3,2
2,0.52,False,2,4,2
3,0.52,False,2,5,2
4,0.52,False,2,6,2
...,...,...,...,...,...
2023,1.00,True,14,10,7
2024,1.00,True,14,11,7
2025,1.00,True,14,12,7
2026,1.00,True,14,13,7


In [175]:
prob_win_df['should_take'] = prob_win_df['probability_take'] > .5

In [176]:
only_takes = prob_win_df[prob_win_df['should_take']].sort_values('probability_take')

In [177]:
only_takes.loc[only_takes.groupby(['players'])['probability_take'].idxmin().dropna()]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
0,0.520000,False,2,2,2,True
468,0.514286,False,12,2,3,True
867,0.504082,True,3,11,4,True
1251,0.535866,True,7,5,5,True
1604,0.512568,True,8,7,6,True
1944,0.512568,True,8,9,7,True


In [178]:
only_takes[only_takes['players'] == 4][0:10]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
867,0.504082,True,3,11,4,True
871,0.504082,True,4,2,4,True
870,0.504082,True,3,14,4,True
869,0.504082,True,3,13,4,True
868,0.504082,True,3,12,4,True
866,0.504082,True,3,10,4,True
865,0.504082,True,3,9,4,True
864,0.504082,True,3,8,4,True
863,0.504082,True,3,7,4,True
862,0.504082,True,3,6,4,True


In [179]:
only_takes[only_takes['players'] == 5][0:10]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
1251,0.535866,True,7,5,5,True
1250,0.535866,True,7,4,5,True
1242,0.535866,True,6,9,5,True
1249,0.535866,True,7,3,5,True
1248,0.535866,True,7,2,5,True
1247,0.535866,True,6,14,5,True
1246,0.535866,True,6,13,5,True
1245,0.535866,True,6,12,5,True
1244,0.535866,True,6,11,5,True
1243,0.535866,True,6,10,5,True


In [180]:
only_takes[only_takes['players'] == 6][0:10]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
1604,0.512568,True,8,7,6,True
1605,0.512568,True,8,8,6,True
1592,0.512568,True,7,8,6,True
1593,0.512568,True,7,9,6,True
1594,0.512568,True,7,10,6,True
1595,0.512568,True,7,11,6,True
1596,0.512568,True,7,12,6,True
1598,0.512568,True,7,14,6,True
1599,0.512568,True,8,2,6,True
1600,0.512568,True,8,3,6,True


In [181]:
only_takes[only_takes['players'] == 7][0:10]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
1944,0.512568,True,8,9,7,True
1955,0.512568,True,9,7,7,True
1954,0.512568,True,9,6,7,True
1953,0.512568,True,9,5,7,True
1952,0.512568,True,9,4,7,True
1956,0.512568,True,9,8,7,True
1949,0.512568,True,8,14,7,True
1950,0.512568,True,9,2,7,True
1948,0.512568,True,8,13,7,True
1947,0.512568,True,8,12,7,True


In [182]:
three_player_takes[~three_player_takes['is_trump']]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
408,0.514286,False,12,2,3,True
409,0.514286,False,12,3,3,True
410,0.514286,False,12,4,3,True
411,0.514286,False,12,5,3,True
412,0.514286,False,12,6,3,True
413,0.514286,False,12,7,3,True
414,0.514286,False,12,8,3,True
415,0.514286,False,12,9,3,True
416,0.514286,False,12,10,3,True
417,0.514286,False,12,11,3,True


In [183]:
three_player_takes[three_player_takes['is_trump']]

,probability_take,is_trump,card_value,trump_card_val,players,should_take
432,0.573878,True,2,2,3,True
433,0.604898,True,2,3,3,True
434,0.604898,True,2,4,3,True
435,0.604898,True,2,5,3,True
436,0.604898,True,2,6,3,True
...,...,...,...,...,...,...
571,0.960000,True,13,9,3,True
572,0.960000,True,13,10,3,True
573,0.960000,True,13,11,3,True
574,0.960000,True,13,12,3,True
